In [1]:
from torch.autograd import Variable
import torch
import pickle
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import evaluation_vectorization as ev
import random
import config

sys.path.append(os.path.join(os.getcwd(), 'Aging_Model'))
import FigureConfig as FC

# Device

In [2]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cuda:0')
device = 'cpu'
device

'cpu'

# Dataset

In [3]:
if config.current_dataset is not None:
    ds = config.datasets[config.current_dataset]
else:
    ds = config.datasets[0]

ds

'Seeds'

In [4]:
datapath = os.path.join(config.path, f'Dataset_{ds}.p')
with open(datapath, 'rb') as f:
    dataset = pickle.load(f)
X = dataset['X'].float()
y = dataset['y']
E, N_features, N_class = X.shape[0], X.shape[1], torch.max(
    torch.unique(y)).item()+1
X.shape, y.shape, E, N_features, N_class

(torch.Size([210, 7]), torch.Size([210]), 210, 7, 3)

## Pseudo-electrical Signal

In [5]:
# normalization
X = X - torch.min(X, axis=0)[0]
X = X / torch.max(X, axis=0)[0]
torch.min(X), torch.max(X)

(tensor(0.), tensor(1.))

In [6]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

# generate tensordataset
dataset = TensorDataset(X.to(device), y.to(device))

# split
train_rate = 0.6
valid_rate = 0.2
test_rate = 0.2
E_train = min(8096, int(E*train_rate))
E_valid = min(2048, int(E*valid_rate))
E_test = min(2048, int(E*test_rate))

random.seed(config.data_split_seed);
np.random.seed(config.data_split_seed);
torch.manual_seed(config.data_split_seed);

train_data, rest_data = random_split(dataset, [E_train, E-E_train])
valid_data, rest_data = random_split(rest_data, [E_valid, E-E_train-E_valid])
test_data, rest_data = random_split(rest_data, [E_test, E-E_train-E_valid-E_test])

len(train_data), len(valid_data), len(test_data)

(126, 42, 42)

In [7]:
# batch
train_loader = DataLoader(train_data, batch_size=len(train_data))
valid_loader = DataLoader(valid_data, batch_size=len(valid_data))
test_loader = DataLoader(test_data, batch_size=len(test_data))

# Evaluation

## Load Models

In [8]:
accs = []
losses = []

for seed in range(10):
    with open(f'./result/topology/6{config.Topology}3/{ds}_PNN_{seed}.p', 'rb') as f:
        PNN = pickle.load(f)
    PNN.to(device)
    
    random.seed(0);
    np.random.seed(0);
    torch.manual_seed(0);

    structure = ev.GetStructure(PNN)
    
    mean_acc_PNN, std_acc_PNN, mean_maa_PNN, std_maa_PNN, loss = ev.Evaluation(PNN, valid_loader, config.M_test, int(config.M_test/10), config.K_test, device)
    accs.append(np.mean(mean_acc_PNN))
    losses.append(loss)
    print(loss)

  0%|          | 0/10 [00:00<?, ?it/s]

0.0651492103934288


  0%|          | 0/10 [00:00<?, ?it/s]

0.06682627648115158


  0%|          | 0/10 [00:00<?, ?it/s]

...

0.0780029371380806


  0%|          | 0/10 [00:00<?, ?it/s]

0.053644631057977676


  0%|          | 0/10 [00:00<?, ?it/s]

0.09367106854915619


In [9]:
best_seed = np.argmin(losses)
best_seed

8

In [10]:
losses[best_seed]

0.053644631057977676

In [11]:
with open(f'./result/topology/6{config.Topology}3/{ds}_PNN_{best_seed}.p', 'rb') as f:
    PNN = pickle.load(f)
PNN.to(device)

Sequential(
  (0): PNNLayer()
  (1): PNNLayer()
)

In [12]:
mean_acc_PNN, std_acc_PNN, mean_maa_PNN, std_maa_PNN, loss = ev.Evaluation(PNN, test_loader, config.M_test, int(config.M_test/10), config.K_test, device)

  0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
loss

0.0988769605755806

In [14]:
np.savetxt(f'./result/topology/6{config.Topology}3/{ds}_PNN_ACC_{best_seed}_{int(np.mean(mean_acc_PNN)*1000)}.txt', np.vstack((mean_acc_PNN, std_acc_PNN)))
np.savetxt(f'./result/topology/6{config.Topology}3/{ds}_PNN_MAA_{best_seed}_{int(np.mean(mean_maa_PNN)*1000)}.txt', np.vstack((mean_maa_PNN, std_maa_PNN)))

In [15]:
mean_maa_PNN, std_maa_PNN

(array([0.9523849 , 0.9278556 , 0.88771284, 0.8385706 , 0.791952  ,
        0.75295234, 0.72161883, 0.699857  , 0.6835715 , 0.6721429 ,
        0.66333336, 0.6582378 , 0.65623784, 0.65433335, 0.65476227,
        0.65566695, 0.6571432 , 0.6595717 , 0.6615237 , 0.66338074,
        0.66590494, 0.66790503, 0.6696668 , 0.6712855 , 0.67414284,
        0.675524  , 0.676762  , 0.67780954, 0.6782382 , 0.67819065,
        0.678143  , 0.6785241 , 0.6787142 , 0.67919016, 0.68004715,
        0.68080926, 0.6814759 , 0.6816187 , 0.6821902 , 0.6829997 ,
        0.6830948 , 0.6831902 , 0.68366617, 0.6843806 , 0.6851899 ,
        0.6849517 , 0.68518984, 0.6848565 , 0.6852848 , 0.685761  ],
       dtype=float32),
 array([3.9339066e-06, 2.4059355e-02, 6.3296288e-02, 1.0067675e-01,
        1.2958466e-01, 1.5043893e-01, 1.6330130e-01, 1.7116177e-01,
        1.7614204e-01, 1.7842372e-01, 1.7924704e-01, 1.7987885e-01,
        1.7970443e-01, 1.8106157e-01, 1.8083017e-01, 1.8071501e-01,
        1.8094613e-01, 1

In [16]:
if losses[best_seed]==0:
    if loss==0:
        overfitting = 1
    else:
        overfitting = 10000
else:
    overfitting = loss/losses[best_seed]
np.savetxt(f'./result/topology/6{config.Topology}3/{ds}_PNN_loss.txt', np.array([losses[best_seed], loss, overfitting]), fmt='%.10f')

In [17]:
overfitting

1.8431846510178629